In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
metadata = {'metadata_col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'categorical', 'na_percentage': 0.0},
                'pir': {'type': 'categorical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [7]:
pm = PETsARD.HyperProcessor(metadata)

In [8]:
from PETsARD.Processor.Missingist import *
from PETsARD.Processor.Outlierist import *
from PETsARD.Processor.Encoder import *
from PETsARD.Processor.Scaler import *

In [9]:
# df.loc[[1,2,3,4,5], 'gen'] = np.nan
# df.loc[[10,20,30,40,5], 'age'] = np.nan

In [10]:
# test global outlierist transformation

config_1 = {'missingist': {'gen': Missingist_Drop(), 'age': Missingist_Mean()},
            'outlierist': {'gen': None, 'age': Outlierist_ZScore()},
            'encoder': {'gen': Encoder_Label()},
            'scaler': {'age': Scaler_Log()}}

In [11]:
pm.update_config(config_1)

In [12]:
pm.get_config()

{'missingist': {'gen': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376f6800>,
  'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1376a7370>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376a73d0>,
  'edu': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376a7460>,
  'mar': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376a6770>,
  'bmi': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1376a6920>,
  'dep': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376f4790>,
  'pir': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376f5840>,
  'gh': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1376f5990>,
  'mets': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1376f7340>,
  'qm': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376f7190>,
  'dia': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1376f70a0>},
 'outlierist': {'gen': None,
  'age': <PETsARD.Processor.Outlierist.Outlierist_ZScore at 0x105c83760>,
  'race': None,
  

In [13]:
pm.fit(df)

In [14]:
df_transformed = pm.transform(df)

In [15]:
df_transformed

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,1,4.127134,0.291068,0.423739,0.314927,-0.202041,0.163945,0.112804,0.0,0.0,0.327598,0.918361
1,1,3.970292,0.004780,0.684146,0.695555,0.235906,0.104438,0.859190,0.0,0.0,0.149174,0.377441
2,1,4.356709,0.162207,0.656944,0.020877,-0.056059,0.558521,0.551294,0.0,0.0,0.767934,0.798891
3,0,4.025352,0.171796,0.491573,0.842031,1.929305,0.976008,0.394018,0.0,0.0,0.902864,0.455598
4,0,3.737670,0.457880,0.249900,0.791406,-1.296911,0.970790,0.500722,0.0,0.0,0.692631,0.643283
...,...,...,...,...,...,...,...,...,...,...,...,...
4061,1,3.218876,0.774935,0.495116,0.551621,-1.194723,0.088677,0.368066,0.0,0.0,0.019685,0.232740
4062,0,4.330733,0.261746,0.720618,0.964549,-1.121732,0.949708,0.425945,0.0,0.0,0.118442,0.738653
4063,0,4.382027,0.099173,0.026462,0.934607,0.265103,0.538829,0.146190,0.0,0.0,0.244051,0.154197
4064,1,3.555348,0.184234,0.790398,0.393390,-0.464810,0.224897,0.966825,0.0,0.0,0.118425,0.036496


In [16]:
df_inverse = pm.inverse_transform(df_transformed)

In [17]:
df_inverse

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,61.999996,White,Graduate,Married,27.799999,0,0,0.0,0.0,Q2,1
1,Male,52.999996,White,HighSchool,Divorced,30.799999,0,1,0.0,0.0,Q1,0
2,Male,78.000015,White,HighSchool,Married,28.799999,0,0,0.0,0.0,Q3,1
3,Female,55.999992,White,Graduate,Parther,42.400002,1,0,0.0,0.0,Q3,0
4,Female,42.000004,Black,College,Divorced,20.299999,1,0,0.0,0.0,Q4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4061,Male,25.000002,Other,Graduate,Never,21.000000,0,0,0.0,0.0,Q1,0
4062,Female,76.000000,White,HighSchool,Widowed,21.500000,1,0,0.0,0.0,Q1,0
4063,Female,80.000000,White,College,Widowed,31.000000,0,0,0.0,0.0,Q1,0
4064,Male,35.000004,White,9th,Married,26.000000,0,1,0.0,0.0,Q1,0


In [18]:
try:
    print(f'Inverse successful:\n{(df_inverse == df).all()}')
    print(f'Numeric close (age): {np.isclose(df_inverse.age, df.age).all()}')
    print(f'Numeric close (bmi): {np.isclose(df_inverse.bmi, df.bmi).all()}')
except:
    print('Not applicable.')

Not applicable.


In [19]:
loader2 = PETsARD.Loader(filepath = '[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader2.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [20]:
df2 = loader2.data

In [21]:
metadata2 = {'metadata_col': {
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'workclass': {'type': 'categorical', 'na_percentage': 0.057307},
                'fnlwgt': {'type': 'numerical', 'na_percentage': 0.0},
                'education': {'type': 'categorical', 'na_percentage': 0.0},
                'educational-num': {'type': 'numerical', 'na_percentage': 0.0},
                'marital-status': {'type': 'categorical', 'na_percentage': 0.0},
                'occupation': {'type': 'categorical', 'na_percentage': 0.057512},
                'relationship': {'type': 'categorical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'gender': {'type': 'categorical', 'na_percentage': 0.0},
                'capital-gain': {'type': 'numerical', 'na_percentage': 0.0},
                'capital-loss': {'type': 'numerical', 'na_percentage': 0.0},
                'hours-per-week': {'type': 'numerical', 'na_percentage': 0.0},
                'native-country': {'type': 'categorical', 'na_percentage': 0.017546},
                'income': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 48842,
                 'col_num': 15,
                 'na_percentage': 0.07411653904426518
                }
            }

In [22]:
pm2 = PETsARD.HyperProcessor(metadata2)

In [23]:
pm2.get_config()

{'missingist': {'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x13788bc70>,
  'workclass': <PETsARD.Processor.Missingist.Missingist_Drop at 0x13788b5b0>,
  'fnlwgt': <PETsARD.Processor.Missingist.Missingist_Mean at 0x13788b4c0>,
  'education': <PETsARD.Processor.Missingist.Missingist_Drop at 0x13788b2e0>,
  'educational-num': <PETsARD.Processor.Missingist.Missingist_Mean at 0x13788a890>,
  'marital-status': <PETsARD.Processor.Missingist.Missingist_Drop at 0x13788a740>,
  'occupation': <PETsARD.Processor.Missingist.Missingist_Drop at 0x13788ae60>,
  'relationship': <PETsARD.Processor.Missingist.Missingist_Drop at 0x13788ae00>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x13788ab00>,
  'gender': <PETsARD.Processor.Missingist.Missingist_Drop at 0x13788a6b0>,
  'capital-gain': <PETsARD.Processor.Missingist.Missingist_Mean at 0x137889cc0>,
  'capital-loss': <PETsARD.Processor.Missingist.Missingist_Mean at 0x13788a290>,
  'hours-per-week': <PETsARD.Processor.Missing

In [24]:
pm2.fit(df2)

In [25]:
df_transformed2 = pm2.transform(df2)

In [26]:
df_transformed2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.995129,0.453547,0.351675,0.835703,-1.197259,0.538591,0.752154,0.705463,0.868790,0.211329,-0.144804,-0.217127,-0.034087,0.661413,0.003695
1,-0.046942,0.062532,-0.945524,0.208791,-0.419335,0.005468,0.916426,0.135749,0.160778,0.109748,-0.144804,-0.217127,0.772930,0.238242,0.278438
2,-0.776316,0.829269,1.394723,0.862423,0.747550,0.264084,0.992529,0.403513,0.664805,0.270859,-0.144804,-0.217127,-0.034087,0.811246,0.958638
3,-1.068066,0.347295,1.704525,0.507826,-0.030373,0.712271,0.737091,0.881287,0.051518,0.787025,-0.144804,-0.217127,-0.034087,0.044484,0.245591
4,-0.192816,0.988784,0.215911,0.648125,1.136512,0.399733,0.410740,0.363486,0.413728,0.474858,-0.144804,-0.217127,-0.034087,0.378704,0.579716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,1.047121,0.073392,1.251867,0.751692,1.525474,0.450565,0.301139,0.112190,0.324968,0.485625,-0.144804,-0.217127,-0.034087,0.224368,0.821404
26929,-1.213941,0.609035,1.140952,0.395685,-0.030373,0.502521,0.985975,0.641524,0.536253,0.055870,-0.144804,-0.217127,-0.034087,0.110987,0.009450
26930,-0.849254,0.330182,0.640492,0.860610,0.747550,0.404900,0.958524,0.926797,0.191965,0.900421,-0.144804,-0.217127,-0.195490,0.531613,0.723362
26931,0.098933,0.107376,-0.334178,0.204527,-0.419335,0.355078,0.800880,0.274327,0.146439,0.087886,-0.144804,-0.217127,-0.034087,0.099146,0.880137


In [27]:
df_inverse2 = pm2.inverse_transform(df_transformed2)

In [28]:
df_inverse2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,24.0,Private,369667.0,Some-college,10.0,Never-married,Other-service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
4,36.0,Federal-gov,212465.0,Bachelors,13.0,Married-civ-spouse,Adm-clerical,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,53.0,Private,321865.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
26929,22.0,NaN,310152.0,Some-college,10.0,Never-married,Protective-serv,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
26930,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
26931,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [29]:
df2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
